In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston

# Configuracion de graficos
plt.style.use('seaborn')
sns.set_palette("husl")

#  Cargar los datos
boston = load_boston()
datos = pd.DataFrame(boston.data, columns=boston.feature_names)
datos['VALOR_CASA'] = boston.target  # Precio de la vivienda

#  Explorar los datos
print("\nPrimeras filas:")
print(datos.head())

print("\nResumen estadístico:")
print(datos.describe())

# Gráficos importantes
plt.figure(figsize=(10, 6))
sns.histplot(datos['VALOR_CASA'], bins=30, kde=True)
plt.title('Distribución de precios de viviendas')
plt.xlabel('Precio (en miles $)')
plt.ylabel('Cantidad')
plt.show()

# Relación de variables
variables = ['RM', 'LSTAT', 'PTRATIO']
for var in variables:
    plt.figure(figsize=(8, 5))
    sns.scatterplot(x=datos[var], y=datos['VALOR_CASA'])
    plt.title(f'Relación entre {var} y el precio')
    plt.show()

# Correlación
correlaciones = datos.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlaciones[['VALOR_CASA']].sort_values('VALOR_CASA', ascending=False), 
            annot=True, cmap='coolwarm')
plt.title('Correlación con el precio de viviendas')
plt.show()

print("\nHallazgos principales:")
print("- A más habitaciones (RM), mayor precio de la vivienda")
print("- En zonas con más población de bajos recursos (LSTAT), los precios bajan")
print("- Donde hay más alumnos por profesor (PTRATIO), los precios son menores")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Cargar dataset desde OpenML
from sklearn.datasets import fetch_openml
boston = fetch_openml(name="boston", version=1, as_frame=True)
data = boston.frame


sns.set(style="whitegrid", palette="pastel")
plt.rcParams["figure.figsize"] = (10, 6)

# 1. Boxplot del valor medio de las viviendas
plt.figure()
sns.boxplot(y=data['MEDV'])
plt.title('Distribución del valor medio de las viviendas')
plt.ylabel('Valor medio (en miles de dólares)')
plt.show()
print("Comentario 1: La mayoría de viviendas se ubica entre 20k y 25k, aunque hay algunas mucho más caras.")

# 2. Frecuencia de propiedades
plt.figure()
data['CHAS'].value_counts().sort_index().plot(kind='bar')
plt.title('¿La propiedad está junto al río Charles?')
plt.xlabel('0 = No | 1 = Sí')
plt.ylabel('Cantidad de propiedades')
plt.xticks(rotation=0)
plt.show()
print("Comentario 2: Muy pocas viviendas están cerca del río Charles, la mayoría no lo está.")

# 3. Boxplot del valor de viviendas por antigüedad 
data['GRUPO_EDAD'] = pd.cut(data['AGE'], bins=[0, 35, 70, 100], labels=['<=35 años', '35-70 años', '70+ años'])

plt.figure()
sns.boxplot(x='GRUPO_EDAD', y='MEDV', data=data)
plt.title('Valor de viviendas según antigüedad del inmueble')
plt.xlabel('Antigüedad del inmueble')
plt.ylabel('Valor medio (en miles de dólares)')
plt.show()
print("Comentario 3: Las casas más viejas (70+ años) suelen valer un poco menos.")

# 4. Dispersión entre contaminación y zona industrial
plt.figure()
sns.scatterplot(x='INDUS', y='NOX', data=data)
plt.title('Contaminación vs zona industrial')
plt.xlabel('Zona industrial (INDUS)')
plt.ylabel('Contaminación (NOX)')
plt.show()
print("Comentario 4: Se nota una relación directa: a más zona industrial, más contaminación.")

# 5. Histograma del ratio alumno-profesor
plt.figure()
sns.histplot(data['PTRATIO'], bins=15, kde=True)
plt.title('Distribución del ratio alumno-profesor')
plt.xlabel('Alumnos por profesor')
plt.ylabel('Frecuencia')
plt.show()
print("Comentario 5: La mayoría de ciudades tienen entre 18 y 20 alumnos por profesor.")


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.datasets import fetch_openml

# Cargar los datos 
boston = fetch_openml(name="boston", version=1, as_frame=True)
df = boston.frame

#  Prueba T: ¿Las casas cerca del río valen más?
casas_rio = df[df['CHAS'] == 1]['MEDV']
casas_no_rio = df[df['CHAS'] == 0]['MEDV']

t_test = stats.ttest_ind(casas_rio, casas_no_rio, equal_var=False)
print("\n1. Prueba T - Casas cerca del río vs no:")
print(f"Precio medio cerca río: ${casas_rio.mean():.2f}k")
print(f"Precio medio lejos del río: ${casas_no_rio.mean():.2f}k")
print(f"¿Son diferentes? p-value = {t_test.pvalue:.4f}")
print("Conclusión: Sí, las casas cerca del río valen significativamente más" if t_test.pvalue < 0.05 else "Conclusión: No hay diferencia significativa")

# 2. ANOVA: ¿El precio varía con la antigüedad de la casa?
df['GRUPO_EDAD'] = pd.cut(df['AGE'], bins=[0, 35, 70, 100], labels=['Nuevas', 'Medianas', 'Viejas'])

modelo_anova = ols('MEDV ~ GRUPO_EDAD', data=df).fit()
anova = sm.stats.anova_lm(modelo_anova)
print("\n2. ANOVA - Precio por antigüedad:")
print(anova)
print("Conclusión: Hay diferencias entre grupos" if anova['PR(>F)'][0] < 0.05 else "Conclusión: No hay diferencias")

# 3. Correlación: ¿Más zonas comerciales = más contaminación?
corr, p_val = stats.pearsonr(df['INDUS'], df['NOX'])
print("\n3. Correlación NOX vs INDUS:")
print(f"Correlación: {corr:.2f} (p-value = {p_val:.4f})")
print("Conclusión: Sí, hay relación positiva fuerte" if p_val < 0.05 else "Conclusión: No hay relación")

# 4. Regresión: ¿Afecta la distancia al trabajo al precio?
X = sm.add_constant(df['DIS'])
modelo = sm.OLS(df['MEDV'], X).fit()
print("\n4. Regresión - Distancia vs Precio:")
print(f"Por cada unidad de distancia, el precio cambia en ${modelo.params[1]:.2f}k")
print("¿Es significativo? p-value =", modelo.pvalues[1])
print("Conclusión: Sí, más distancia aumenta el precio" if modelo.pvalues[1] < 0.05 else "Conclusión: No afecta significativamente")

# Gráficos 
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.boxplot([casas_rio, casas_no_rio], labels=['Junto al río', 'No junto'])
plt.title('Precio por cercanía al río')

plt.subplot(2, 2, 2)
df.groupby('GRUPO_EDAD')['MEDV'].mean().plot(kind='bar')
plt.title('Precio medio por antigüedad')

plt.subplot(2, 2, 3)
plt.scatter(df['INDUS'], df['NOX'])
plt.title('Contaminación vs Zonas comerciales')

plt.subplot(2, 2, 4)
plt.scatter(df['DIS'], df['MEDV'])
plt.title('Precio vs Distancia a trabajos')

plt.tight_layout()
plt.show()
